# Preprocessing Best Books Dataset
1. Dropping columns we don't need
2. Checking for duplicates
3. Converting to the correct data type
4. Handling missing values
5. Splitting multi-labeled columns
6. Removing typos and standardizing names

Python 3.11.8

In [1]:
# GENERAL LIBRARIES
import numpy as np
import pandas as pd
import re
import kaggle_dataset # .py file
import difflib

In [2]:
print(' numpy:',np.__version__,'\n', 'pandas:',pd.__version__,'\n', 're:',re.__version__)

 numpy: 1.26.4 
 pandas: 2.1.4 
 re: 2.2.1


### Import the data

In [3]:
datasets_path = kaggle_dataset.list_of_kaggle_datasets('https://www.kaggle.com/datasets/justinnguyen0x0x/best-books-of-the-21st-century-dataset')
raw_data = pd.read_csv(datasets_path[0], index_col = "id")
raw_data.head()

Note: You need to provide your Kaggle API credentials
Skipping, found downloaded files in "./best-books-of-the-21st-century-dataset" (use force=True to force download)
Files downloaded:
best-books-of-the-21st-century-dataset/Best_Book_21st.csv


,title,series,author,book_link,genre,date_published,publisher,num_of_page,lang,review_count,rating_count,rate,award
id,,,,,,,,,,,,,
0,The Martian,\n (The Martian #1)\n,Andy Weir,https://www.goodreads.com/book/show/18007564-t...,"Science Fiction,Fiction,Audiobook,Adventure,Sp...",February 11th 2014,"Crown\n\n ,",384.0,English,"77,156","919,781",4.40,Kurd-Laßwitz-Preis for Bestes ausländisches We...
1,Under the Banner of Heaven: A Story of Violent...,NaN,Jon Krakauer,https://www.goodreads.com/book/show/10847.Unde...,"Nonfiction,Religion,History,Crime,True Crime,M...",2004,"Pan MacMillan\n\n ,",400.0,English,"10,981","173,895",3.98,Colorado Book Award for General Nonfiction (2004)
2,Cutting for Stone,NaN,Abraham Verghese,https://www.goodreads.com/book/show/3591262-cu...,"Fiction,Historical,Historical Fiction,Cultural...",February 3rd 2009,"Alfred A. Knopf\n\n ,",541.0,English,"26,253","353,365",4.29,"PEN/Hemingway Foundation Award Nominee (2010),..."
3,We Need to Talk About Kevin,NaN,Lionel Shriver,https://www.goodreads.com/book/show/80660.We_N...,"Fiction,Contemporary,Thriller,Horror,Mystery,C...",July 3rd 2006,"Harper Perennial\n\n ,",400.0,English,"14,787","187,639",4.04,Orange Prize for Fiction (2005)
4,The Immortal Life of Henrietta Lacks,NaN,Rebecca Skloot,https://www.goodreads.com/book/show/6493208-th...,"Nonfiction,Science,History,Biography,Health,Me...",February 2nd 2010,"Crown Publishing Group\n\n ,",370.0,English,"36,574","646,155",4.07,Ambassador Book Award for American Studies (20...


## 1. Dropping columns we don't need

In [4]:
# For each section, we will have a checkpoint, in case we need to go back in the code for modifications
df = raw_data.copy()
# We will drop the following columns:
# - book_link: This is not relevant information to our future analysis, since it's only informative
# - num_of_page: This is not relevant information to our future analysis, since it's only informative
# - review_count: There is no further data on reviews that can help us in our future analysis. It's only informative
df = df.drop(['book_link', 'num_of_page', 'review_count'], axis = 1)
df.head()

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
id,,,,,,,,,,
0,The Martian,\n (The Martian #1)\n,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...",February 11th 2014,"Crown\n\n ,",English,"919,781",4.40,Kurd-Laßwitz-Preis for Bestes ausländisches We...
1,Under the Banner of Heaven: A Story of Violent...,NaN,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...",2004,"Pan MacMillan\n\n ,",English,"173,895",3.98,Colorado Book Award for General Nonfiction (2004)
2,Cutting for Stone,NaN,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...",February 3rd 2009,"Alfred A. Knopf\n\n ,",English,"353,365",4.29,"PEN/Hemingway Foundation Award Nominee (2010),..."
3,We Need to Talk About Kevin,NaN,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...",July 3rd 2006,"Harper Perennial\n\n ,",English,"187,639",4.04,Orange Prize for Fiction (2005)
4,The Immortal Life of Henrietta Lacks,NaN,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...",February 2nd 2010,"Crown Publishing Group\n\n ,",English,"646,155",4.07,Ambassador Book Award for American Studies (20...


## 2. Checking for duplicates

In [5]:
df_duplicates_removed = df.copy()
# Retrieving a subset of the dataset with duplicated title and author
non_duplicates = df_duplicates_removed.drop_duplicates(subset = ['title','author']).index
df_duplicates_removed.loc[~df_duplicates_removed.index.isin(non_duplicates),:]

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
id,,,,,,,,,,
900,I Am Malala: The Story of the Girl Who Stood U...,NaN,"Malala Yousafzai,Christina Lamb","Nonfiction,Biography,Autobiography,Memoir,Femi...",October 8th 2013,"Little, Brown and Company\n\n ,",English,"489,281",4.12,"Lincoln Award Nominee (2017),Australian Book I..."
901,Carnal: Somewhere Over 40 Winks,NaN,Rom LcO'Feer,"Contemporary,Psychology,Fiction",September 16th 2009,"Lulu.com\n\n ,",English,"4,859",3.47,NaN
902,The Unlikely Pilgrimage of Harold Fry (Harold ...,\n (Harold Fry #1)\n,Rachel Joyce,"Fiction,Contemporary,Audiobook,Book Club,Adult...",July 24th 2012,"Random House\n\n ,",English,"162,236",3.91,"Booker Prize Nominee for Longlist (2012),Desmo..."
903,"Legend (Legend, #1)",\n (Legend #1)\n,Marie Lu,"Young Adult,Science Fiction,Dystopia,Science F...",November 29th 2011,Putnam Juvenile,English,"467,828",4.16,"Rhode Island Teen Book Award Nominee (2013),Mi..."
904,Dreamcatcher,NaN,Stephen King,"Horror,Fiction,Science Fiction,Thriller,Fantas...","March 1st 2002 ,",NaN,English,"162,211",3.64,NaN
...,...,...,...,...,...,...,...,...,...,...
1816,Amazonia,NaN,James Rollins,"Thriller,Adventure,Fiction,Mystery,Action,Scie...",March 1st 2002,Avon Books,English,"32,815",4.13,NaN
1822,Together: AI and Human. On The Same Side.,NaN,Zoltan Andrejkovics,"Artificial Intelligence,Futurism,Pop Culture,G...",September 27th 2019,Amazon Digital Services LLC,English,50,4.18,NaN
4231,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",May 19th 2016,"Black Swan\n\n ,",English,"253,935",3.89,British Book Award for The Author of the Year ...


In [6]:
# Exploring a few of the duplicated values. Note: This line was run with different duplicated indices to get a general
# idea of what issues we can find in the dataset
index_to_check = 4231
df_duplicates_removed[(df_duplicates_removed['title'] == df_duplicates_removed.iloc[index_to_check, 0]) & 
                      (df_duplicates_removed['author'] == df_duplicates_removed.iloc[index_to_check, 2])]

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
id,,,,,,,,,,
768,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",October 18th 2006,"Houghton Mifflin Co. (Boston/NY)\n\n ,",English,"253,491",3.89,British Book Award for The Author of the Year ...
1357,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",October 18th 2006,"Houghton Mifflin Co. (Boston/NY)\n\n ,",English,"253,930",3.89,British Book Award for The Author of the Year ...
4231,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",May 19th 2016,"Black Swan\n\n ,",English,"253,935",3.89,British Book Award for The Author of the Year ...


In [7]:
# 1. Most of the duplicates seem to be updates of the same book and author
# 2. There are a few exceptions regarding the same book with different date_published and different publisher
#    which we will review individually
df_duplicates_removed = df_duplicates_removed.drop_duplicates(subset = ['title', 'author', 'date_published', 'publisher'],
                                          keep = 'last')

In [8]:
# Let's review the special cases we have left
non_duplicates = df_duplicates_removed.drop_duplicates(subset = ['title','author'], keep = False).index
df_duplicates_removed.loc[~df_duplicates_removed.index.isin(non_duplicates),:].sort_values(by = 'title')

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
id,,,,,,,,,,
570,"A Dog's Purpose (A Dog's Purpose, #1)",\n (A Dog's Purpose #1)\n,W. Bruce Cameron,"Fiction,Animals,Animals,Dogs,Contemporary,Adul...",July 2010,Forge Books,English,"121,815",4.33,Goodreads Choice Award Nominee for Fiction (20...
1000,"A Dog's Purpose (A Dog's Purpose, #1)",\n (A Dog's Purpose #1)\n,W. Bruce Cameron,"Fiction,Animals,Animals,Dogs,Contemporary,Adul...",July 2010,Forge,English,"122,008",4.33,Goodreads Choice Award Nominee for Fiction (20...
6359,Norse Mythology,NaN,Neil Gaiman,"Fantasy,Fantasy,Mythology,Fiction,Audiobook,Sh...",February 7th 2017,W. W. Norton & Company,English,"261,139",4.08,"Tähtifantasia Award Nominee (2020),Goodreads C..."
8973,Norse Mythology,NaN,Neil Gaiman,"Fantasy,Fantasy,Mythology,Fiction,Audiobook,Sh...",March 6th 2018,"W. W. Norton & Company\n\n ,",English,"261,110",4.08,"Tähtifantasia Award Nominee (2020),Goodreads C..."
62,Saving CeeCee Honeycutt,NaN,"Beth Hoffman,Jenna Lamia","Fiction,Historical,Historical Fiction,Womens F...",January 12th 2010,"Penguin Audio\n\n ,",English,"82,797",3.92,Goodreads Choice Award Nominee for Fiction and...
1065,Saving CeeCee Honeycutt,NaN,"Beth Hoffman,Jenna Lamia","Fiction,Historical,Historical Fiction,Womens F...",January 12th 2010,Penguin Audio,English,"82,868",3.92,Goodreads Choice Award Nominee for Fiction and...
1357,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",October 18th 2006,"Houghton Mifflin Co. (Boston/NY)\n\n ,",English,"253,930",3.89,British Book Award for The Author of the Year ...
4231,The God Delusion,NaN,Richard Dawkins,"Nonfiction,Religion,Science,Philosophy,Religio...",May 19th 2016,"Black Swan\n\n ,",English,"253,935",3.89,British Book Award for The Author of the Year ...


In [9]:
# There are four special duplicate cases left:
# 1. Two of them look like typos in the publisher, so we can keep the most updated review history (last record)
# 2. The other two have different publish date. This may be because the publisher published a new version of the same book
#    (in the first case), and because there was a publisher that published the book first (second case). In both cases, we
#    want to keep the first date that the book was published (first record)
df_duplicates_removed = df_duplicates_removed.drop_duplicates(subset = ['title', 'author', 'date_published'],
                                          keep = 'last') # First issue
df_duplicates_removed = df_duplicates_removed.drop_duplicates(subset = ['title', 'author'], keep = 'first') # Second issue

In [10]:
# Reseting the indexes
df_duplicates_removed = df_duplicates_removed.reset_index(drop = True)
# Let's verify all duplicates for title and author were removed
df_duplicates_removed.duplicated(subset = ['title', 'author'], keep = False).sum()

0

## 3. Converting to the correct data type

In [11]:
df_correct_dtype = df_duplicates_removed.copy()
# Let's check what is the date type of each column
df_correct_dtype.dtypes

title              object
series             object
author             object
genre              object
date_published     object
publisher          object
lang               object
rating_count       object
rate              float64
award              object
dtype: object

**String columns: 'title', 'author', 'genre', 'publisher', 'lang'**

In [12]:
# First, we convert all the string columns to a string data type
columns_to_convert = ['title', 'author', 'genre', 'publisher', 'lang']
df_correct_dtype[columns_to_convert] = df_correct_dtype[columns_to_convert].astype(dtype = 'string')
df_correct_dtype.dtypes

title             string[python]
series                    object
author            string[python]
genre             string[python]
date_published            object
publisher         string[python]
lang              string[python]
rating_count              object
rate                     float64
award                     object
dtype: object

**Integer columns: 'rating_count', 'award'**

In [13]:
# Columns 'rating_count', 'award' should be integer columns
# 1. 'award' must be transformed to numbers
# Mapping books with no awards
no_awards = df_correct_dtype['award'].isnull()
# Iterating through the awards to count them
for row in range(df_correct_dtype.shape[0]):
    if no_awards[row] == True:
        df_correct_dtype.loc[row, 'award'] = 0
    else:
        df_correct_dtype.loc[row, 'award'] = len(df_correct_dtype.loc[row, 'award'].split(","))

In [14]:
# 2. We must remove "," from 'rating_count'
for row in range(df_correct_dtype.shape[0]):
    if type(df_correct_dtype.loc[row, 'rating_count']) == str:
        df_correct_dtype.loc[row, 'rating_count'] = df_correct_dtype.loc[row, 'rating_count'].replace(",", "")
            
# Also, NaN values mean there are no ratings
df_correct_dtype.loc[:, ['rating_count']] = df_correct_dtype.loc[:, ['rating_count']].fillna(0)

In [15]:
# Transform the columns to integer type
columns_to_convert = ['rating_count', 'award']
df_correct_dtype[columns_to_convert] = df_correct_dtype[columns_to_convert].astype(dtype = 'int32')
df_correct_dtype.dtypes

title             string[python]
series                    object
author            string[python]
genre             string[python]
date_published            object
publisher         string[python]
lang              string[python]
rating_count               int32
rate                     float64
award                      int32
dtype: object

**Boolean columns: 'series'**

In [16]:
# All values from 'series' that are null means they don't belong to a series of books. Therefore, they must be set to 0
df_correct_dtype.loc[:, 'series'] = df_correct_dtype.loc[:, 'series'].fillna(0)
# All the others belong to a series of books, and must be set to 1
for row in range(df_correct_dtype.shape[0]):
    if df_correct_dtype.loc[row, 'series'] != 0:
        df_correct_dtype.loc[row, 'series'] = 1

In [17]:
# We convert the 'series' column to boolean
columns_to_convert = ['series']
df_correct_dtype[columns_to_convert] = df_correct_dtype[columns_to_convert].astype(dtype = 'bool')
df_correct_dtype.dtypes

title             string[python]
series                      bool
author            string[python]
genre             string[python]
date_published            object
publisher         string[python]
lang              string[python]
rating_count               int32
rate                     float64
award                      int32
dtype: object

**Float columns: date_published**

In [18]:
# Since for some dates we have the full date, for some we have month and year, and for some only the year, we will keep
# only the year in all columns. Also, when we don't have a date available we cannot transform it to 0. It will need to
# stay as a null value. Due to this, we cannot transform the data type to integer, and we will keep it as floating
for row in range(df_correct_dtype.shape[0]):
    if type(df_correct_dtype.loc[row, 'date_published']) == str:
        if re.findall('[1-9][0-9]{3}', df_correct_dtype.loc[row, 'date_published']):
            df_correct_dtype.loc[row, 'date_published'] = re.findall('[1-9][0-9]{3}', df_correct_dtype.loc[row, 'date_published'])[0]
        else:
            df_correct_dtype.loc[row, 'date_published'] = np.NaN

In [19]:
# We transform the 'date_published' column to float:
columns_to_convert = 'date_published'
df_correct_dtype[columns_to_convert] = df_correct_dtype[columns_to_convert].astype(dtype = 'float')
df_correct_dtype.dtypes

title             string[python]
series                      bool
author            string[python]
genre             string[python]
date_published           float64
publisher         string[python]
lang              string[python]
rating_count               int32
rate                     float64
award                      int32
dtype: object

## 4. Handling missing values

In [20]:
df_nans_removed = df_correct_dtype.copy()
# We will first take a look at the total of null values per columns. We will handle each column depending on what we want
df_nans_removed.isnull().sum()

title               0
series              0
author             15
genre             979
date_published    145
publisher         380
lang              507
rating_count        0
rate               15
award               0
dtype: int64

**Missing values: 'rating_count', 'rate'**

In [21]:
# We will start with the missing values for our target variables, since we need these values for our analysis
df_nans_removed[df_nans_removed[['rating_count', 'rate']].isnull().any(axis = 1)]

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
0,The Known World,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
1,Coraline,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
2,"Graceling (Graceling Realm, #1)",False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
3,Year of Wonders,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
4,The Last Olympian (Percy Jackson and the Olymp...,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
5,Freedom,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
6,"A Feast for Crows (A Song of Ice and Fire, #4)",False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
7,The Goldfinch,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
8,The Sea of Monsters (Percy Jackson and the Oly...,False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0
9,"The Maze Runner (The Maze Runner, #1)",False,<NA>,<NA>,NaN,<NA>,<NA>,0,NaN,0


In [22]:
# We can see that the only value we have is the title of the book. Therefore, we can remove these records
df_nans_removed = df_nans_removed.dropna(subset = ['rating_count', 'rate'])
# We check how our null values table looks now
df_nans_removed.isnull().sum()

title               0
series              0
author              0
genre             964
date_published    130
publisher         365
lang              492
rating_count        0
rate                0
award               0
dtype: int64

In [23]:
# Also, we can deduce that if the rating_count is 0, then no one has evaluated the book, and the rate becomes irrelevant
df_nans_removed[df_nans_removed['rating_count'] == 0]

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
3214,STORY OF AN INDIAN WOMAN,False,Anantaroopa,<NA>,2015.0,<NA>,English,0,0.0,0
3494,Don’t Monkey Around When It Comes To Saving Mo...,False,D. Terry,<NA>,2013.0,BookTango,English,0,0.0,0
3716,You Win Even When You Lose: 50 Secrets And Tip...,False,D. Terry,<NA>,2013.0,BookTango,English,0,0.0,0
3959,Pretty Near Perfect: : 40 Skin Care Tips To He...,False,D. Terry,<NA>,2013.0,BookTango,English,0,0.0,0
4247,"Six-Pack Abs: A Quick, Concise Guide To Obtain...",False,D. Terry,<NA>,2013.0,BookTango,English,0,0.0,0
7630,Grave Results Book Two: Blood Rites,False,Dana Reed,<NA>,2005.0,"PublishAmerica ,",English,0,0.0,0


In [24]:
# Therefore, we will consider these values as "null" and remove them from the dataset
df_nans_removed = df_nans_removed[df_nans_removed['rating_count'] != 0].reset_index(drop = True)

**Missing values: 'genre', 'publisher', 'lang'**

In [25]:
# For the missing values of 'genre', 'publisher' and 'lang', we can change the null values to Unknown
unknown_values = df_nans_removed.loc[:,['genre', 'publisher', 'lang']].fillna('Unknown')
df_nans_removed.loc[:, ['genre', 'publisher', 'lang']] = unknown_values
df_nans_removed.reset_index(drop = True, inplace = True)
# We check how our null values table looks now. We should only have null values in 'date_published'
# The approach we will take is to replace the date_published with the most common value.
# This will be done during the Data Analysis
df_nans_removed.isnull().sum()

title               0
series              0
author              0
genre               0
date_published    130
publisher           0
lang                0
rating_count        0
rate                0
award               0
dtype: int64

## 5. Splitting multi-labeled columns

In [26]:
df_label_splitted = df_nans_removed.copy()

In [27]:
# We will keep only the first genre that appears for each book (the main one)
# Since we already dealt with the missing values, we can expect to have at least one value
for row in range(len(df_label_splitted)):
    genres = df_label_splitted.loc[row,'genre'].split(",")[0:2]
    df_label_splitted.loc[row, 'genre'] = genres[0]
    # NOTE: If we wanted to include two genres, we would have to generate a new record for each book.
    # In that case, we can use the code below within the for loop
    #try:
    #    new_row = df_label_splitted.loc[row, :].copy()
    #    new_row.loc['genre'] = genres[1]
    #    df_label_splitted = df_label_splitted.append(new_row, ignore_index = True)
    #except:
    #    pass

In [28]:
# We will keep only the first author that appears for each book (the main one)
# Since we already dealt with the missing values, we can expect to have at least one value
for row in range(len(df_label_splitted)):
    authors = df_label_splitted.loc[row,'author'].split(",")[0:2]
    df_label_splitted.loc[row, 'author'] = authors[0]
    # NOTE: If we wanted to include two authors, we would have to generate a new record for each book.
    # In that case, we can use the code below within the for loop
    #try:
    #    new_row = df_label_splitted.loc[row, :].copy()
    #    new_row.loc['author'] = genres[1]
    #    df_label_splitted = df_label_splitted.append(new_row, ignore_index = True)
    #except:
    #    pass

In [29]:
df_label_splitted

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award
0,Fantastic Beasts and Where to Find Them,True,Newt Scamander,Fantasy,2001.0,"Arthur A. Levine Books ,",English,372109,3.96,0
1,Houdini Heart,False,Ki Longfellow,Horror,2011.0,Eio Books,English,10409,3.75,0
2,"An Echo in the Bone (Outlander, #7)",True,Diana Gabaldon,Historical,2010.0,"Orion ,",English,133940,4.40,2
3,Three Junes,False,Julia Glass,Fiction,2003.0,"Anchor ,",English,45789,3.56,1
4,"The Dark Tower (The Dark Tower, #7)",True,Stephen King,Fantasy,2006.0,"Pocket Books ,",English,162749,4.23,1
...,...,...,...,...,...,...,...,...,...,...
9112,"Edge of Passion (Stealth Guardians, #1)",True,Tina Folsom,Fantasy,2012.0,Unknown,English,1011,4.01,0
9113,Tales from a Not-So-Popular Party Girl (Dork D...,True,Rachel Renée Russell,Childrens,2010.0,Aladdin,English,54285,4.24,0
9114,Bottoms,False,Tanja Kirschner,Unknown,2005.0,"NorthSouth ,",English,11,3.55,0
9115,The Wedding Dress,False,Gary E. Parker,Unknown,2002.0,"Victor ,",English,23,3.61,0


## 6. Removing typos and standardizing names

In [30]:
df_typos_removed = df_label_splitted.copy()

**Typos: genre**

In [31]:
# Verify genres don't have similar values
genres = df_typos_removed['genre'].unique()
for i in range(len(genres)):
    close_matches = difflib.get_close_matches(genres[i],genres[i+1:], n = 1, cutoff = 0.7)
    if len(close_matches) != 0:
        print(genres[i],':', close_matches)

# NOTE: No typos were found. Also, genres cannot be so easily standardize, even if they are very similar.
# For example, a Historical book (perhaps a historical novel based on a historical fact) is different to a History book

Historical : ['History']
Fiction : ['Fan Fiction']
Science Fiction : ['Science Fiction Fantasy']
Womens Fiction : ['Fan Fiction']
Autobiography : ['Biography']
Christian : ['Christianity']
Cultural : ['Culture']
Christian Fiction : ['Christianity']
Psychology : ['Mythology']
Health : ['Death']
Eastern Africa : ['Northern Africa']
Literature : ['Asian Literature']
European Literature : ['Asian Literature']


**Typos: author**

In [32]:
# Verify authors don't have similar values
authors = df_typos_removed['author'].unique()
for i in range(len(authors)):
    close_matches = difflib.get_close_matches(authors[i],authors[i+1:], cutoff = 0.92)
    if len(close_matches) != 0:
        print(authors[i],':', close_matches)

Matthew Quick : ['Matthew Quirk']
Michelle Moran : ['Michelle  Morgan']
Steven Erikson : ['Steve Erickson']
Richard  Holmes : ['Richard   Holmes']
Elizabeth Berg : ['Elizabeth Bear']
Emmanuel M. Kolini : ['Emmanual M. Kolini']
RamG Vallath : ['Ram G. Vallath']
Victor G. LaVanWay : ['Victor G. La Van Way']
David Abram : ['David Abrams']
Justin Richardson : ['Justin Richards']
Alycia  Ripley : ['Alycia Ripley']
Richard Zimler : ['Richard Bimler']


In [33]:
# These are the only authors with real typos (the others were verified to be different authors):
    # Richard Holmes
    # Emmanuel M. Kolini
    # Ram G. Vallath
    # Victor G. La Van Way
    # Alycia Ripley
# Since it's possible for the dataset to be updated (and have new errors), they will be fixed completely manually
df_typos_removed.loc[(df_typos_removed['author'] == 'Richard  Holmes') | (df_typos_removed['author'] == 'Richard   Holmes'),'author'] = 'Richard Holmes'
df_typos_removed.loc[df_typos_removed['author'] == 'Emmanual M. Kolini','author'] = 'Emmanuel M. Kolini'
df_typos_removed.loc[df_typos_removed['author'] == 'RamG Vallath','author'] = 'Ram G. Vallath'
df_typos_removed.loc[df_typos_removed['author'] == 'Victor G. LaVanWay','author'] = 'Victor G. La Van Way'
df_typos_removed.loc[df_typos_removed['author'] == 'Alycia  Ripley','author'] = 'Alycia Ripley'

**Typos: publisher**

In [34]:
# We will remove new lines, commas and front and back whitespaces from the publisher
print('BEFORE:\n',df_typos_removed['publisher'].head())
df_typos_removed['publisher'] = df_typos_removed['publisher'].str.replace('\n','').str.replace(',','').str.strip()
print('\nAFTER:\n',df_typos_removed['publisher'].head())

BEFORE:
 0    Arthur A. Levine Books

            ,
1                                Eio Books
2                     Orion

            ,
3                    Anchor

            ,
4              Pocket Books

            ,
Name: publisher, dtype: string

AFTER:
 0    Arthur A. Levine Books
1                 Eio Books
2                     Orion
3                    Anchor
4              Pocket Books
Name: publisher, dtype: string


In [35]:
# Verify publishers don't have similar values (1st verification)
publishers = df_typos_removed['publisher'].unique()
for i in range(len(publishers)):
    close_matches = difflib.get_close_matches(publishers[i],publishers[i+1:], cutoff=0.9)
    if len(close_matches) != 0:
        print(publishers[i],':', close_matches)

Little Brown and Company : ['Little Brown And Company', 'Little Brown & Company']
W. W. Norton  Company : ['W. W. Norton & Company', 'W.W. Norton & Company']
Simon & Schuster : ['Simon  Schuster', 'Simon Schuster', 'Simon & Schuster UK']
Bloomsbury Publishing PLC : ['Bloomsbury Publishing Plc', 'Bloomsbury Publishing']
Farrar Straus and Giroux : ['Farrar Straus Giroux', 'Farrar Straus and Giroux (NY)']
G.P. Putnam's Sons : ["G. P. Putnam's Sons", 'G.P. Putnam’s Sons']
Farrar Straus and Giroux (NY) : ['Farrar Straus and Giroux (BYR)', 'Farrar Straus and Giroux (Byr)']
Bloomsbury USA Childrens : ["Bloomsbury USA Children's", 'Bloomsbury Childrens']
Simon & Schuster Books for Young Readers : ['Simon  Schuster Books for Young Readers', 'Simon & Schuster Books For Young Readers', 'Simon Schuster Books for Young Readers']
St. Martin's Press : ['St. Martins Press', "St Martin's Press", 'St. Martin’s Press']
Disney-Hyperion : ['Disney/Hyperion', 'Disney Hyperion']
Scholastic : ['scholastic']
S

In [36]:
# At a first glance, there are several issues that can be standardize in order to have a more standard publisher
# NOTE: A new column will be created, since we only need unique values for the analysis and we will be able to go back to the original publisher name
# 1. Lower case all of the publishers
df_typos_removed['publisher_clean'] = df_typos_removed['publisher'].str.lower()
# 2. Remove & and "and"
df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(' & ',' ').str.replace(' and ',' ')
# 3. Remove special characters (letters only)
df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace('[^a-z\s]+',' ',regex = True).str.strip()
# 4. Remove non-neceessary abbreviations at the middle and at end of the name
df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(' co ',' ').str.replace(' ltd ',' ').str.replace(' inc ',' ')
strings_to_remove = [' co$',' ltd$',' inc$',' llc$',' plc$',' ya$',' ink$']
for remove in strings_to_remove:
    df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(remove, '', regex = True).str.strip()
# 5. Remove countries/cities at the middle and at the end of the name
df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(' usa ',' ')
places_to_remove = [' uk$',' us$',' usa$',' ny$',' nyc$',' byr$',' gb$',' au$',' ma$',' az$']
for remove in places_to_remove:
    df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(remove, '', regex = True).str.strip()
# 6. Remove all whitespaces
df_typos_removed['publisher_clean'] = df_typos_removed['publisher_clean'].str.replace(' ','')

In [37]:
# Verify publishers don't have similar values (2nd verification)
publishers = df_typos_removed['publisher_clean'].unique()
# NOTE: There are many different publishers that end with "publishing", so we will remove them from the checklist
publishers = list(filter(lambda x: not x.endswith('publishing'),publishers))
list_of_corrections = []
for i in range(len(publishers)):
    close_matches = difflib.get_close_matches(publishers[i],publishers[i+1:], cutoff=0.86)
    if len(close_matches) != 0:
        list_of_corrections.append([publishers[i],close_matches])
        print(publishers[i],':', close_matches)

eiobooks : ['keibooks']
simonschuster : ['simonschusterbfyr']
margaretkmcelderrybooks : ['margaretkmcelderry']
penguinpress : ['thepenguinpress']
bloomsburychildrens : ['bloomsburychildrensbooks']
alfredaknopf : ['alfredknopf']
stmartinspress : ['stmartinspresspl']
delacortepress : ['delacortepr']
houghtonmifflinharcourt : ['houghtonmifflinharcourtboston']
thechickenhouse : ['chickenhouse']
torbooks : ['orbooks']
duttonbooksforyoungreaders : ['crownbooksforyoungreaders']
harpercollins : ['harpercoll', 'harpercollinsecco', 'eccoharpercollins']
canongateus : ['canongate']
createspaceindependentpublishingplatform : ['createspaceindpendentpublishingplatform']
marinerbooks : ['warnerbooks']
littlebrownbooksforyoungreaders : ['littlebrowncompanybooksforyoungreaders']
randomhouseaudio : ['randomhouseaudiobooks']
emuhousepublications : ['dreamhousepublication']
bloomsburychildrensbooks : ['bloomsburyusachildrensbooks']
penguingroup : ['thepenguingroup', 'penguingroupdaw']
hyperionbookch : ['hy

In [38]:
# After reviewing several cutoff percentages, it was determined 0.86 was the best option, with a few cases that should be removed from the list:
remove_from_corrections = [['eiobooks', ['keibooks']],
                           ['torbooks', ['orbooks']],
                           ['duttonbooksforyoungreaders', ['crownbooksforyoungreaders']],
                           ['marinerbooks', ['warnerbooks']],
                           ['emuhousepublications', ['dreamhousepublication']],
                           ['zolabooks', ['zolandbooks']],
                           ['randomhouseaudiopublishinggroup',['randomhousepublishinggroup', 'therandomhousepublishinggroup']],
                           ['atheneumbooksforyoungreaders', ['hmhbooksforyoungreaders']],
                           ['knopfbooksforyoungreaders',['alfredaknopfbooksforyoungreaders', 'vikingbooksforyoungreaders']],
                           ['greenwillowbooks', ['greenhillbooks']],
                           ['owlbooks', ['howbooks']],
                           ['salmonpress', ['almondpress']],
                           ['oakpress', ['akpress']],
                           ['oxforduniversitypress', ['samforduniversitypress', 'fordhamuniversitypress', 'stanforduniversitypress']],
                           ['sfischer', ['fischer']], #Different standardization
                           ['recordedbooks', ['colorecordedbooks']],
                           ['pokrovpublications', ['oakgrovepublications']],
                           ['audiobooks', ['bbcaudiobooks']],
                           ['kjcbooks', ['jcpbooks']],
                           ['amanapublications', ['vmapublications']],
                           ['universityofarizonapress', ['universityofiowapress']],
                           ['universityofnorthcarolinapress', ['universityofsouthcarolinapress']],
                           ['querido', ['emquerido']],
                           ['fordhamuniversitypress', ['samforduniversitypress']],
                           ['nationalgeographickids', ['nationalgeographic']]
                           ]
for remove in remove_from_corrections:
    list_of_corrections.remove(remove)
# Adding the cases with "Different standardization"
list_of_corrections.append(['fischer', ['sfischer']])

In [39]:
# Standardize remaining list in publisher_clean column
for i in list_of_corrections:
    df_typos_removed.loc[df_typos_removed['publisher_clean'].isin(i[1]),'publisher_clean'] = i[0]

# Final dataset saved to CSV

In [40]:
df_final = df_typos_removed.copy()
df_final

,title,series,author,genre,date_published,publisher,lang,rating_count,rate,award,publisher_clean
0,Fantastic Beasts and Where to Find Them,True,Newt Scamander,Fantasy,2001.0,Arthur A. Levine Books,English,372109,3.96,0,arthuralevinebooks
1,Houdini Heart,False,Ki Longfellow,Horror,2011.0,Eio Books,English,10409,3.75,0,eiobooks
2,"An Echo in the Bone (Outlander, #7)",True,Diana Gabaldon,Historical,2010.0,Orion,English,133940,4.40,2,orion
3,Three Junes,False,Julia Glass,Fiction,2003.0,Anchor,English,45789,3.56,1,anchor
4,"The Dark Tower (The Dark Tower, #7)",True,Stephen King,Fantasy,2006.0,Pocket Books,English,162749,4.23,1,pocketbooks
...,...,...,...,...,...,...,...,...,...,...,...
9112,"Edge of Passion (Stealth Guardians, #1)",True,Tina Folsom,Fantasy,2012.0,Unknown,English,1011,4.01,0,unknown
9113,Tales from a Not-So-Popular Party Girl (Dork D...,True,Rachel Renée Russell,Childrens,2010.0,Aladdin,English,54285,4.24,0,aladdin
9114,Bottoms,False,Tanja Kirschner,Unknown,2005.0,NorthSouth,English,11,3.55,0,northsouth
9115,The Wedding Dress,False,Gary E. Parker,Unknown,2002.0,Victor,English,23,3.61,0,victor


In [41]:
df_final.to_csv('Best_Book_21st_clean_dataset.csv', index = False)